In [4]:
import numpy as np
import pandas as pd


#df = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/4hrs/BTC_combined_2504_08v4.csv')

df = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/4hrs/BTC_combined_2024_v2.csv')

df['bar_time'] = pd.to_datetime(df['bar_time'])

df = df.set_index('bar_time')

df = df.drop(columns=[c for c in df.columns if c.lower().startswith('unnamed')], errors='ignore')

df['funding_rate_&'] = df['funding_rate'] * 100


df


,spot_close,spot_vol,prep_close,perp_vol,funding_rate,funding_rate_&
bar_time,,,,,,
2024-01-01 00:00:00+00:00,42330.49,3948.08335,42369.8,30275.835,0.000374,0.037409
2024-01-01 04:00:00+00:00,42492.46,2706.99880,42527.0,18884.162,0.000374,0.037409
2024-01-01 08:00:00+00:00,42690.20,2948.80274,42727.6,21225.402,0.000272,0.027213
2024-01-01 12:00:00+00:00,42783.05,2936.44406,42818.0,23160.827,0.000272,0.027213
2024-01-01 16:00:00+00:00,43517.99,5686.97164,43556.2,46470.443,0.000336,0.033601
...,...,...,...,...,...,...
2025-08-31 04:00:00+00:00,108708.26,1177.25929,108656.9,9506.115,-0.000012,-0.001232
2025-08-31 08:00:00+00:00,108389.76,1590.89462,108350.0,12792.960,0.000047,0.004749
2025-08-31 12:00:00+00:00,108818.69,2441.88905,108758.2,10880.915,0.000047,0.004749


In [5]:
PRICE = "prep_close"
FUND  = "funding_rate"
CAP   = 0.0001    # ~0.01%
HIGH_N = 12
LOW_N  = 18       # ~3 days on 4H bars
TAKER  = 0.0004   # taker fee rate per side (0.04%)
NOTIONAL = 10_000  # fixed USD notional per trade
INCLUDE_FUNDING_CF = True

# ====== PREP & SIGNALS (build if not already) ======
bt = df.copy()
if not pd.api.types.is_datetime64_any_dtype(bt.index):
    bt.index = pd.to_datetime(bt.index, utc=True)
bt = bt.sort_index()

bt["roll_high"] = bt[PRICE].rolling(HIGH_N, min_periods=HIGH_N).max()
bt["fund_lowN"] = bt[FUND].rolling(LOW_N,  min_periods=LOW_N).min()

bt["sig_top"]    = (bt[FUND] >= CAP - 1e-12) & (bt[PRICE] >= bt["roll_high"])
bt["sig_bottom"] = (bt[FUND] <= bt["fund_lowN"])

bt["sig_top_edge"]    = bt["sig_top"] & (~bt["sig_top"].shift(1).fillna(False))
bt["sig_bottom_edge"] = bt["sig_bottom"] & (~bt["sig_bottom"].shift(1).fillna(False))

# ====== BACKTEST (flip on opposite signal, next-bar execution) ======
trades = []
pos = 0              # +1 long, -1 short, 0 flat
qty = 0.0            # contracts, sized by notional at entry
entry_px = np.nan
entry_time = None
fund_acc_usd = 0.0   # funding cashflow in USD

idx = bt.index
N = len(bt)

def px_at(i):  # you can switch to 'open' if you have it
    return float(bt.iloc[i][PRICE])

for i in range(N - 1):  # need i+1 to execute
    want_long  = bool(bt.iloc[i]["sig_bottom_edge"])
    want_short = bool(bt.iloc[i]["sig_top_edge"])

    if pos == 0:
        if want_long ^ want_short:
            side = +1 if want_long else -1
            px = px_at(i+1)
            pos = side
            entry_px = px
            entry_time = idx[i+1]
            qty = (NOTIONAL / px) * pos                 # quantity sized to $1k notional
            fund_acc_usd = 0.0
            # entry fee (USD)
            entry_fee_usd = TAKER * NOTIONAL
            # we’ll attach fees to the trade on exit
            pending_entry_fee = entry_fee_usd
        continue

    # accumulate funding cashflow (approx: rate * notional * sign)
    if INCLUDE_FUNDING_CF and pd.notna(bt.iloc[i][FUND]):
        fund_acc_usd += float(bt.iloc[i][FUND]) * NOTIONAL * pos

    flip_to_long  = (pos == -1) and want_long
    flip_to_short = (pos == +1) and want_short
    if flip_to_long or flip_to_short:
        # exit current at next bar
        exit_px = px_at(i+1)
        exit_time = idx[i+1]

        # price PnL in USD = qty * (exit - entry)
        pnl_price_usd = qty * (exit_px - entry_px)
        pnl_funding_usd = fund_acc_usd
        exit_fee_usd = TAKER * NOTIONAL
        pnl_usd = pnl_price_usd + pnl_funding_usd - (pending_entry_fee + exit_fee_usd)

        trades.append({
            "entry_time": entry_time,
            "side": "LONG" if pos == 1 else "SHORT",
            "entry_price": entry_px,
            "exit_time": exit_time,
            "exit_price": exit_px,
            "bars_held": (bt.index.get_loc(exit_time) - bt.index.get_loc(entry_time)),
            "pnl_price_usd": pnl_price_usd,
            "funding_usd": pnl_funding_usd,
            "fees_usd": -(pending_entry_fee + exit_fee_usd),
            "pnl_total_usd": pnl_usd,
        })

        # flip: open new opposite at the same next bar
        new_side = +1 if flip_to_long else -1
        px_new = exit_px
        pos = new_side
        entry_px = px_new
        entry_time = exit_time
        qty = (NOTIONAL / px_new) * pos
        fund_acc_usd = 0.0
        pending_entry_fee = TAKER * NOTIONAL  # entry fee for new trade

# Close any open trade at last bar
if pos != 0:
    exit_px = px_at(N-1)
    exit_time = idx[N-1]
    pnl_price_usd = qty * (exit_px - entry_px)
    pnl_funding_usd = fund_acc_usd
    exit_fee_usd = TAKER * NOTIONAL
    pnl_usd = pnl_price_usd + pnl_funding_usd - (pending_entry_fee + exit_fee_usd)
    trades.append({
        "entry_time": entry_time,
        "side": "LONG" if pos == 1 else "SHORT",
        "entry_price": entry_px,
        "exit_time": exit_time,
        "exit_price": exit_px,
        "bars_held": (bt.index.get_loc(exit_time) - bt.index.get_loc(entry_time)),
        "pnl_price_usd": pnl_price_usd,
        "funding_usd": pnl_funding_usd,
        "fees_usd": -(pending_entry_fee + exit_fee_usd),
        "pnl_total_usd": pnl_usd,
    })

log = pd.DataFrame(trades).sort_values("entry_time")
log["cum_pnl_usd"] = log["pnl_total_usd"].cumsum()

summary = {
    "trades": int(len(log)),
    "win_rate": float((log["pnl_total_usd"] > 0).mean()) if len(log) else np.nan,
    "avg_trade_usd": float(log["pnl_total_usd"].mean()) if len(log) else np.nan,
    "median_trade_usd": float(log["pnl_total_usd"].median()) if len(log) else np.nan,
    "total_pnl_usd": float(log["pnl_total_usd"].sum()) if len(log) else 0.0,
    "max_dd_usd": float((log["cum_pnl_usd"].cummax() - log["cum_pnl_usd"]).max()) if len(log) else 0.0,
}

print(summary)
print(log.head(12))
print(log.'p')

/var/folders/kz/cfb6kmcd0j5g728cbyjlwwvw0000gp/T/ipykernel_37245/3824327983.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bt["sig_top_edge"]    = bt["sig_top"] & (~bt["sig_top"].shift(1).fillna(False))
/var/folders/kz/cfb6kmcd0j5g728cbyjlwwvw0000gp/T/ipykernel_37245/3824327983.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bt["sig_bottom_edge"] = bt["sig_bottom"] & (~bt["sig_bottom"].shift(1).fillna(False))


{'trades': 115, 'win_rate': 0.5652173913043478, 'avg_trade_usd': -32.10775827140851, 'median_trade_usd': 34.934961352858835, 'total_pnl_usd': -3692.3922012119783, 'max_dd_usd': 7124.893466288244}
                  entry_time   side  entry_price                 exit_time  \
0  2024-01-04 00:00:00+00:00   LONG      43119.6 2024-01-06 00:00:00+00:00   
1  2024-01-06 00:00:00+00:00  SHORT      43925.8 2024-01-27 20:00:00+00:00   
2  2024-01-27 20:00:00+00:00   LONG      42102.2 2024-01-28 04:00:00+00:00   
3  2024-01-28 04:00:00+00:00  SHORT      42369.9 2024-01-28 12:00:00+00:00   
4  2024-01-28 12:00:00+00:00   LONG      42244.5 2024-01-29 20:00:00+00:00   
5  2024-01-29 20:00:00+00:00  SHORT      43287.7 2024-01-31 04:00:00+00:00   
6  2024-01-31 04:00:00+00:00   LONG      43031.6 2024-02-05 12:00:00+00:00   
7  2024-02-05 12:00:00+00:00  SHORT      42629.7 2024-02-06 12:00:00+00:00   
8  2024-02-06 12:00:00+00:00   LONG      43232.0 2024-02-06 20:00:00+00:00   
9  2024-02-06 20:00:00+0

In [10]:
log.groupby("side")[["pnl_price_usd","funding_usd","pnl_total_usd"]].sum()



,pnl_price_usd,funding_usd,pnl_total_usd
side,,,
LONG,3898.601167,1435.511,4870.112167
SHORT,-6509.815368,-1596.689,-8562.504368


In [11]:
log.assign(month=log["entry_time"].dt.to_period("M")).groupby("month")["pnl_total_usd"].sum()

/var/folders/kz/cfb6kmcd0j5g728cbyjlwwvw0000gp/T/ipykernel_37245/2825202387.py:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  log.assign(month=log["entry_time"].dt.to_period("M")).groupby("month")["pnl_total_usd"].sum()


month
2024-01     764.489055
2024-02   -3078.474875
2024-03    -932.836506
2024-04    -739.386987
2024-05     451.219936
2024-06    1209.711140
2024-07     -72.896226
2024-09     -23.619001
2024-10    -551.470366
2024-11   -1580.895734
2024-12   -1073.302591
2025-01     106.198816
2025-05    1212.099034
2025-07    -121.847141
2025-08     738.619247
Freq: M, Name: pnl_total_usd, dtype: float64

In [8]:
log["bars_held"].describe()

count    115.000000
mean      31.608696
std       71.491098
min        1.000000
25%        6.500000
50%       12.000000
75%       29.000000
max      589.000000
Name: bars_held, dtype: float64

In [15]:
cols = ["entry_time","exit_time","side","entry_price","exit_price",
        "bars_held","pnl_price_usd","funding_usd","fees_usd","pnl_total_usd"]
top20 = log.sort_values("pnl_total_usd", ascending=False).head(20)[cols]
print(top20)


                   entry_time                 exit_time   side  entry_price  \
104 2025-05-24 04:00:00+00:00 2025-07-11 04:00:00+00:00   LONG     107550.6   
38  2024-05-15 12:00:00+00:00 2024-05-25 20:00:00+00:00   LONG      64685.5   
51  2024-06-30 12:00:00+00:00 2024-07-09 12:00:00+00:00  SHORT      61722.4   
47  2024-06-09 20:00:00+00:00 2024-06-19 12:00:00+00:00  SHORT      69667.1   
56  2024-07-15 16:00:00+00:00 2024-07-26 04:00:00+00:00   LONG      63309.9   
113 2025-08-22 16:00:00+00:00 2025-08-26 00:00:00+00:00  SHORT     116625.5   
31  2024-03-31 08:00:00+00:00 2024-04-02 04:00:00+00:00  SHORT      70442.6   
35  2024-04-11 04:00:00+00:00 2024-04-13 04:00:00+00:00  SHORT      70789.1   
12  2024-02-08 20:00:00+00:00 2024-02-09 12:00:00+00:00   LONG      45269.1   
81  2024-12-08 20:00:00+00:00 2024-12-10 00:00:00+00:00  SHORT     101200.0   
27  2024-03-21 00:00:00+00:00 2024-03-22 12:00:00+00:00  SHORT      66707.8   
102 2025-05-21 00:00:00+00:00 2025-05-22 04:00:00+00